In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

stop_words = stopwords.words('english')
stop_words.remove('not')
lemmatizer = WordNetLemmatizer()

In [ ]:
contraction_dict = {"dont": "do not", "aint": "is not", "isnt": "is not", "doesnt": "does not", "cant": "cannot", "mustnt": "must not", "hasnt": "has not", "havent": "have not", "arent": "are not", "ain't": "is not", "aren't": "are not","can't": "cannot", "‘cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "Iam": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

# if contraction_dict.has_key('aya'):
#     print('yaya')

def replace_contractions(question):
    return [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in question]

In [ ]:
def qes_preprocessing(qes):
    # Data cleaning:
    qes = re.sub(re.compile('<.*?>'), '', qes)
    qes = re.sub('[^A-Za-z0-9]+', ' ', qes)

    # Lowercase:
    qes = qes.lower()

    # Tokenization:
    tokens = word_tokenize(qes)

    # Contractions replacement:
    tokens = [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in tokens]

    # Stop words removal:
    tokens = [w for w in tokens if w not in stop_words]
    
    # Lemmatization:
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    # Join words after preprocessed:
    qes = ' '.join(tokens) 

    return qes

In [ ]:
from tqdm import tqdm

tqdm.pandas()
train_df['preprocessed_questions'] = train_df['question_text'].progress_apply(qes_preprocessing)

In [ ]:
train_df.info()

In [ ]:
insincere = train_df[train_df['target'] == 0]
sincere = train_df[train_df['target'] == 1]

In [ ]:
len(sincere)

In [ ]:
insincere_batch = insincere[:len(sincere)]

In [ ]:
insincere_batch.shape

In [ ]:
insincere_batch.head()

In [ ]:
test_batch = pd.concat([insincere_batch, sincere])

In [ ]:
test_batch

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,4), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight="balanced", C=0.45, max_iter=250, verbose=1, n_jobs=-1))])

X_train, X_test, y_train, y_test = train_test_split(test_batch['preprocessed_questions'], test_batch.target, test_size=0.2, stratify = test_batch.target.values)

In [ ]:
for i in range(15):
    print('Batch: ', i)
    insincere_batch = insincere[:((i + 1) * len(sincere))]
    test_batch = pd.concat([insincere_batch, sincere])
    
    X_train, X_test, y_train, y_test = train_test_split(test_batch['preprocessed_questions'], test_batch.target, test_size=0.2, stratify = test_batch.target.values)
    
    lr_model = pipeline.fit(X_train, y_train)
    
    get_fscore_matrix(lr_model, 'Linear Regression')

In [ ]:
# lr_model = pipeline.fit(X_train, y_train)

In [ ]:
def get_fscore_matrix(fitted_clf, model_name):
    print(model_name, ' :')
    
    # get classes predictions for the classification report 
    y_train_pred, y_pred = fitted_clf.predict(X_train), fitted_clf.predict(X_test)
    print(classification_report(y_test, y_pred), '\n') # target_names=y
    
    # computes probabilities keep the ones for the positive outcome only      
    print(f'F1-score = {f1_score(y_test, y_pred):.2f}')

In [ ]:
# from sklearn.metrics import f1_score, confusion_matrix, classification_report

# get_fscore_matrix(lr_model, 'Linear Regression')

In [ ]:
test_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
test_df.head()

In [ ]:
test_df['preprocessed'] = test_df['question_text'].apply(qes_preprocessing)

In [ ]:
predictions = lr_model.predict(test_df['preprocessed'])

In [ ]:
print(len(predictions))

In [ ]:
test_df['prediction'] = predictions
results = test_df[['qid', 'prediction']]
results.to_csv('submission.csv', index=False)
results.shape